# 서울시 전기차 충전 데이터 이용

In [2]:
import pandas as pd
df = pd.read_csv(r'D:\charging_data\charging_data_modify.csv', encoding='cp949')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10972 entries, 0 to 10971
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   순번        10972 non-null  int64  
 1   충전소명      10972 non-null  object 
 2   충전일       10972 non-null  object 
 3   충전시작시간    10972 non-null  object 
 4   충전종료시간    10972 non-null  object 
 5   충전량(kWh)  10972 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 514.4+ KB
None


In [3]:
# 쓸모없는 열 삭제
df = df.drop('순번', axis=1)

# 충전소명을 기준으로 데이터 수 확인 
df['충전소명'].value_counts()

충전소명
양재솔라스테이션        2783
동작구청            1211
양천구청             960
홍제1동 공용주차장       688
어린이대공원 후문주차장     651
중랑구청             618
성동구청             537
남부초 공영주차장        476
서대문구청 주차장        449
서소문청사(지상 급속)     265
망우제2 거주자주차장      265
복정역 공영주차장        225
송월동차고지(급속)       218
송월동차고지(완속)       209
마곡중앙광장공영주차장      209
개화산역 공영주차장       183
서울시 본관청사(완속)     173
가산 공영주차장         153
남산청사             153
서소문청사(지하 급속)     138
창동역 공영주차장        111
서소문청사(후생동)        87
서울시 본관청사(급속)      79
서소문청사(완속)         62
천호역 공영주차장         62
중화1동 공영주차장         7
Name: count, dtype: int64

In [3]:
df['충전일자'] = pd.to_datetime(df['충전일'])
df['충전일자'].max() - df['충전일자'].min()


Timedelta('117 days 00:00:00')

In [ ]:
# 충전 세션 500개 이하 삭제
filter = df['충전소명'].value_counts()

# 10000보다 작은 loc 추출
to_drop = filter[filter <= 500].index

# to_drop에 들어있지 않은 loc만 추출
df = df[~df['충전소명'].isin(to_drop)]

# 환경부 충전 데이터 삭제 (지역 확인 어려움)
df = df.drop(df[df['loc'] == '환경부'].index)

df['loc'].value_counts()

In [ ]:
# 충전 시간 (hour) 확인 후 이상치 제거
df['charging_hour'].value_counts()

In [ ]:
# 5시간 이상 충전한 데이터 삭제
df = df[df['charging_hour'] <= 2]

df['charging_hour'].value_counts()




In [ ]:
df = df[df['charging_minute'] >= 5]

df['charging_minute'].value_counts()

In [ ]:
df['loc'].value_counts()

In [ ]:
# 3개월 동안 900 번의 충전 세션이 없는 데이터 삭제 (이상치로 간주, 하루 약 3번 충전)

counts = df['id'].value_counts()

id_keep = counts[counts > 900].index

df = df[df['id'].isin(id_keep)]

df['id'].value_counts()



In [ ]:
# 요일을 0-월 1-화 ... 6-일 로 맵핑 (datetime 객체 이용)

df['date'] = pd.to_datetime(df['date'])

df['date'] = df['date'].dt.weekday

df['date'].value_counts()

In [ ]:
# 주말인지 아닌지 여부 데이터 추가

def is_weekend(day):
    if day in [0,1,2,3,4]:
        return 0
    else:
        return 1

df['weekend'] = df['date'].apply(is_weekend)


In [ ]:
# 하루를 10분 단위의 144개로 나누어, 충전 시작 시간에 맞게 할당
def time_calculate(time):
    hour, minute = time.split()[1].split(':')
    total_minutes = int(hour) * 60 + int(minute)
    time_interval = total_minutes // 10 
    return time_interval

df['time'] = df['start_time'].apply(time_calculate)

In [ ]:
#새로운 df 구성


charging_df = df.iloc[:, [8, 7, 4, 1]]
charging_df = charging_df.drop_duplicates()

# id 별 중복 값이 너무 많은 것인지 id당 3개월에 하루 1세션도 안됨.
charging_df["id"].value_counts()